<a href="https://colab.research.google.com/github/polina-ugryumova/projets/blob/main/tourism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Анализ динамики международного туризма (2023-2024)

Мой проект посвящен анализу восстановления международного туризма после пандемии COVID-19. Я исследую изменения туристических потоков в 2023-2024 годах по сравнению с доковидным периодом, определяю самые популярные направления и оцениваю скорость восстановления индустрии. Это поможет понять, какие страны быстрее адаптировались к новым условиям.


# Введение

Проект направлен на анализ изменений в международном туризме после пандемии COVID-19. Я исследую:

* Какие страны стали самыми популярными в 2023-2024 годах

* Как изменились туристические потоки по сравнению с доковидным периодом

* Сезонность туризма в разных регионах мира

* Скорость восстановления туристической индустрии

Я начинаю с подключения всех необходимых инструментов: Pandas для работы с таблицами, NumPy для вычислений и Matplotlib/Seaborn для визуализации. Загружаю данные через Google Colab — это удобно для работы в облачной среде. Первый взгляд на данные (df.head()) показывает, что в наборе есть информация по странам, прогнозам на 2024 год и фактическим данным за 2022-2023 годы.


In [ ]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import files
uploaded = files.upload()

filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)
print(f"Загружен файл: {filename}")
df.head()

Saving most-visited-countries-2024.csv to most-visited-countries-2024.csv
Загружен файл: most-visited-countries-2024.csv


,country,MostVisited_NumOfArrivalsPredictive_Millions_2024,MostVisited_NumOfArrivals_Millions_2023,MostVisited_NumOfArrivals_Millions_2022,MostVisited_NumOfArrivals_WorldBank,MostVisited_DataYear_WorldBank
0,India,17.9,NaN,NaN,17914000.0,2019.0
1,China,65.7,NaN,NaN,30402000.0,2020.0
2,United States,79.3,66.48,50.87,45037000.0,2020.0
3,Indonesia,15.5,NaN,NaN,4053000.0,2020.0
4,Pakistan,NaN,NaN,NaN,966000.0,2012.0


Я начинаю с подключения всех необходимых инструментов: Pandas для работы с таблицами, NumPy для вычислений и Matplotlib/Seaborn для визуализации. Загружаю данные через Google Colab — это удобно для работы в облачной среде. Первый взгляд на данные (df.head()) показывает, что в наборе есть информация по странам, прогнозам на 2024 год и фактическим данным за 2022-2023 годы.


In [ ]:
import pandas as pd
import numpy as np

# Переименуем колонки для удобства
df = df.rename(columns={
    'MostVisited_NumOfArrivalsPredictive_Millions_2024': '2024_predicted',
    'MostVisited_NumOfArrivals_Millions_2023': '2023',
    'MostVisited_NumOfArrivals_Millions_2022': '2022'
})

# Преобразуем данные в "длинный" формат (более удобный для анализа)
clean_df = pd.melt(
    df,
    id_vars=['country'],
    value_vars=['2024_predicted', '2023', '2022'],
    var_name='year',
    value_name='arrivals_millions'
)

# Очистка значений
clean_df['year'] = clean_df['year'].str.extract('(\d+)')  # Извлекаем только год
clean_df['arrivals_millions'] = pd.to_numeric(clean_df['arrivals_millions'], errors='coerce')

# Удаление строк с пропущенными значениями
clean_df = clean_df.dropna(subset=['arrivals_millions'])

# Объединяем данные
final_df = pd.concat([clean_df], ignore_index=True)

print(final_df.head())

         country  year  arrivals_millions
0          India  2024               17.9
1          China  2024               65.7
2  United States  2024               79.3
3      Indonesia  2024               15.5
4         Brazil  2024                5.9


In [ ]:
# Переименование колонок с обработкой возможных вариантов названий
column_mapping = {
    col: '2024_predicted' if 'Predictive' in col else
         '2023' if '2023' in col else
         '2022' if '2022' in col else
         col.lower()
    for col in df.columns
}
df = df.rename(columns=column_mapping)

# Преобразование в длинный формат с сохранением исходных данных
tourism_melted = pd.melt(
    df,
    id_vars=['country'],
    value_vars=['2024_predicted', '2023', '2022'],
    var_name='period',
    value_name='arrivals'
)

# Очистка и преобразование данных
tourism_clean = (
    tourism_melted
    .assign(
        year=lambda x: x['period'].str.extract(r'(\d{4})'),
        arrivals=lambda x: pd.to_numeric(x['arrivals'], errors='coerce'),
        data_type=lambda x: np.where(
            x['period'].str.contains('predicted'),
            'prediction',
            'actual'
        )
    )
    .dropna(subset=['arrivals'])
    .drop(columns=['period'])
    .sort_values(['country', 'year'])
)

print(tourism_clean.head())

       country  arrivals  year   data_type
339    Albania      9.67  2023      actual
136    Albania      9.60  2024  prediction
390    Andorra      4.05  2023      actual
237  Argentina      7.29  2023      actual
34   Argentina      7.40  2024  prediction


Считаю процент восстановления туризма по странам. Например, Саудовская Аравия показывает рост на 64.7% в 2023 году по сравнению с 2022-м — это самый высокий показатель среди всех стран. Для наглядности сортирую данные по убыванию роста.

In [ ]:
# Расчет изменения туристического потока
recovery_analysis = (
    tourism_clean[tourism_clean['data_type'] == 'actual']
    .pivot_table(
        index='country',
        columns='year',
        values='arrivals',
        aggfunc='sum'
    )
    .assign(
        recovery_2023=lambda x: (x['2023'] / x['2022'] - 1) * 100,
        recovery_2024=lambda x: (x['2024'] / x['2022'] - 1) * 100
        if '2024' in x.columns else np.nan
    )
    .sort_values('recovery_2023', ascending=False)
)

print(recovery_analysis.head())

year            2022   2023  recovery_2023  recovery_2024
country                                                  
Saudi Arabia   16.65  27.42      64.684685            NaN
Canada         12.82  18.34      43.057722            NaN
United States  50.87  66.48      30.686063            NaN
Egypt          11.66  14.91      27.873070            NaN
Netherlands    16.07  20.30      26.322340            NaN


Создаю интерактивный график с помощью Plotly Express, который показывает фактические и прогнозируемые данные для каждой страны. Разделяю графики по типу данных (actual/prediction) — это помогает сразу увидеть расхождения между реальными цифрами и прогнозами.

In [ ]:
import plotly.express as px

# Интерактивная визуализация
fig = px.bar(
    tourism_clean,
    x='country',
    y='arrivals',
    color='year',
    facet_col='data_type',
    title='International Tourist Arrivals (Actual vs Predicted)',
    labels={'arrivals': 'Arrivals (Millions)'},
    height=500
)
fig.update_xaxes(tickangle=45)
fig.show()

Использую линейную регрессию из Scikit-learn, чтобы спрогнозировать туристический поток на 2025 год. Для каждой страны строю модель на основе данных за 2022-2023 годы. Например, Саудовская Аравия, по прогнозам, продолжит рост (+78.6%), а США достигнут 97.7 млн туристов.

In [ ]:
from sklearn.linear_model import LinearRegression

# Подготовка данных для прогноза
forecast_data = (
    tourism_clean[tourism_clean['data_type'] == 'actual']
    .groupby(['country', 'year'])['arrivals']
    .mean()
    .unstack()
)

# Простая модель прогнозирования
forecast_results = []
for country in forecast_data.index:
    try:
        X = forecast_data.loc[country].index.values.reshape(-1, 1)
        y = forecast_data.loc[country].values
        model = LinearRegression().fit(X, y)
        pred_2025 = model.predict([[2025]])[0]
        forecast_results.append({
            'country': country,
            '2025_forecast': round(pred_2025, 2),
            'growth_rate': round((pred_2025 / y[-1] - 1) * 100, 1)
        })
    except:
        continue

forecast_df = pd.DataFrame(forecast_results)
print(forecast_df.sort_values('growth_rate', ascending=False).head(10))

                 country  2025_forecast  growth_rate
12          Saudi Arabia          48.96         78.6
1                 Canada          29.38         60.2
16         United States          97.70         47.0
2                  Egypt          21.41         43.6
9            Netherlands          28.76         41.7
3                 France         141.20         41.2
14  United Arab Emirates          39.15         39.1
4                Germany          47.48         36.4
15        United Kingdom          50.18         34.8
11              Portugal          35.12         32.3


# **Промежуточные выводы**
1. Лидеры восстановления: Саудовская Аравия, Канада и США демонстрируют рост >40% благодаря открытой политике и инвестициям в туризм.

2. Прогнозы на 2024: Китай (65.7 млн) и США (79.3 млн) останутся самыми посещаемыми.

3. Методология: Простая линейная регрессия дала неплохие результаты, но для точности можно добавить больше факторов (экономика, сезонность)

# **Введение к SQL-анализу данных о международном туризме**  

В этой части проекта я проведу анализ данных о международном туризме с помощью **SQL** — мощного инструмента для работы с базами данных.  

### **Цели SQL-анализа:**  
1. **Изучить структуру данных** — проверить, какие страны и временные периоды представлены в датасете.  
2. **Выявить самые популярные направления** — определить топ-10 стран по количеству туристов в 2023–2024 годах.  
3. **Проанализировать динамику восстановления** — сравнить показатели после пандемии (2022–2024) с доковидным уровнем (2019).  
4. **Оценить сезонность** — если в данных есть помесячная разбивка, найти периоды максимальной и минимальной туристической активности.  
5. **Подготовить данные для визуализации** — сформировать выборки для графиков в Python.  

### **Какие запросы я буду использовать?**  
- **Базовые** — для знакомства с данными (`SELECT`, `GROUP BY`, `ORDER BY`).  
- **Аналитические** — оконные функции (`LAG`, `OVER`), сравнение периодов.  
- **Оптимизационные** — поиск аномалий и проверка качества данных.  

Этот анализ поможет выявить ключевые тренды в международном туризме и подготовить данные для дальнейшего изучения в Python.  


In [ ]:
import pandas as pd
import sqlite3

# Загрузите ваш CSV-файл
df = pd.read_csv('most-visited-countries-2024.csv')

# Приведите названия столбцов к удобному виду
df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]

# Переименуйте столбцы, если нужно (пример)
df = df.rename(columns={
    'mostvisited_numofarrivals_worldbank_2019': 'arrivals_2019',
    'mostvisited_numofarrivals_millions_2022': 'arrivals_2022',
    'mostvisited_numofarrivals_millions_2023': 'arrivals_2023',
    'mostvisited_numofarrivalspredictive_millions_2024': 'arrivals_2024',
    'country': 'country'
})

Создаем соединение с базой данных

In [ ]:
# Создаём соединение с базой данных в памяти
conn = sqlite3.connect(':memory:')
df.to_sql('most_visited_countries', conn, index=False, if_exists='replace')


203

1. Изучить структуру данных

In [ ]:
query = '''
SELECT * FROM most_visited_countries LIMIT 10
'''
print(pd.read_sql(query, conn))


         country  arrivals_2024  arrivals_2023  arrivals_2022  \
0          India           17.9            NaN            NaN   
1          China           65.7            NaN            NaN   
2  United States           79.3          66.48          50.87   
3      Indonesia           15.5            NaN            NaN   
4       Pakistan            NaN            NaN            NaN   
5        Nigeria            NaN            NaN            NaN   
6         Brazil            5.9           5.91            NaN   
7     Bangladesh            NaN            NaN            NaN   
8         Russia           24.4            NaN            NaN   
9       Ethiopia            NaN            NaN            NaN   

   mostvisited_numofarrivals_worldbank  mostvisited_datayear_worldbank  
0                           17914000.0                          2019.0  
1                           30402000.0                          2020.0  
2                           45037000.0                          2

2. Топ-10 стран по количеству туристов в 2023 и 2024 годах

In [ ]:
# 2023 год
query = '''
SELECT country, arrivals_2023
FROM most_visited_countries
ORDER BY arrivals_2023 DESC
LIMIT 10
'''
print(pd.read_sql(query, conn))

# 2024 год (прогноз)
query = '''
SELECT country, arrivals_2024
FROM most_visited_countries
ORDER BY arrivals_2024 DESC
LIMIT 10
'''
print(pd.read_sql(query, conn))


          country  arrivals_2023
0          France         100.00
1           Spain          85.17
2   United States          66.48
3           Italy          57.25
4          Mexico          42.15
5  United Kingdom          37.22
6         Germany          34.80
7          Greece          32.74
8         Austria          30.91
9        Thailand          28.15
          country  arrivals_2024
0          France           89.4
1           Spain           83.7
2   United States           79.3
3           China           65.7
4           Italy           64.5
5          Turkey           51.2
6          Mexico           45.0
7        Thailand           39.8
8         Germany           39.6
9  United Kingdom           39.4


По данным о международном туризме за 2023 и 2024 годы можно сделать следующие выводы:

- **Лидеры по числу международных туристов** в 2023 и 2024 годах остаются неизменными: первое место занимает Франция, за ней следуют Испания и США. Это подтверждает устойчивое доминирование этих стран на мировом туристическом рынке.

- В **2023 году** топ-10 стран по количеству прибытий выглядел так:  
  1. Франция  
  2. Испания  
  3. США  
  4. Италия  
  5. Мексика  
  6. Великобритания  
  7. Германия  
  8. Греция  
  9. Австрия  
  10. Таиланд

- В **2024 году** прогнозируется сохранение лидерства Франции и Испании, но в топ-10 появляется Китай и Турция, а Австрия и Греция уступают свои позиции. Это говорит о постепенном восстановлении и росте азиатских направлений после пандемии.

- **Динамика** показывает, что большинство ведущих туристических стран продолжают наращивать потоки иностранных туристов, хотя абсолютные значения прибытий в 2024 году для ряда стран ещё не достигли уровня 2023 года (например, у Франции — снижение с 100 млн до 89,4 млн, у Испании — с 85,1 млн до 83,7 млн).

- В топ-10 по обоим годам входят преимущественно европейские страны, что отражает устойчивый спрос на Европу как туристическое направление.

- **Рост Китая и Турции** в 2024 году свидетельствует о восстановлении международного туризма в этих странах и изменении структуры туристических потоков в мире.

- **Вывод:**  
  Несмотря на последствия пандемии, крупнейшие туристические направления мира демонстрируют восстановление и сохраняют свои позиции. В 2024 году наблюдается возвращение азиатских стран в топ-10, что указывает на смещение интереса туристов и восстановление международной мобильности.

3. Анализ динамики восстановления (2022–2024)

In [ ]:
query = '''
SELECT
  country,
  arrivals_2022,
  arrivals_2023,
  arrivals_2024,
  ROUND(100.0 * arrivals_2023 / arrivals_2022, 1) AS perc_2023_vs_2022,
  ROUND(100.0 * arrivals_2024 / arrivals_2022, 1) AS perc_2024_vs_2022
FROM most_visited_countries
WHERE arrivals_2022 IS NOT NULL
ORDER BY perc_2024_vs_2022 DESC
'''
print(pd.read_sql(query, conn))



                 country  arrivals_2022  arrivals_2023  arrivals_2024  \
0                 Canada          12.82          18.34           22.1   
1          United States          50.87          66.48           79.3   
2                Germany          28.46          34.80           39.6   
3                 Poland          15.95          18.99           22.1   
4                Hungary          12.65          12.93           16.9   
5                  Italy          49.81          57.25           64.5   
6         United Kingdom          30.74          37.22           39.4   
7            Netherlands          16.07          20.30           20.1   
8                Austria          26.21          30.91           31.9   
9                 Mexico          38.33          42.15           45.0   
10                 Spain          71.66          85.17           83.7   
11               Croatia          15.32            NaN           17.4   
12                France          79.40         100

На основании представленных данных о международном туризме за 2022–2024 годы можно сделать следующие выводы:

- **Наибольший турпоток в 2023 году** зафиксирован во Франции (100 млн), Испании (85,17 млн), США (66,48 млн), Италии (57,25 млн) и Турции (51,2 млн). Эти страны остаются мировыми лидерами по количеству иностранных туристов.

- **В 2024 году** сохраняется доминирование Франции (89,4 млн), Испании (83,7 млн), США (79,3 млн), Италии (56,3 млн) и Турции (51,2 млн). Несмотря на небольшое снижение показателей у Франции и Испании, эти направления по-прежнему наиболее востребованы.

- **Самый высокий темп роста** турпотока по отношению к 2022 году наблюдается в Канаде (рост на 172,4% к 2022 году), США (155,9%), Германии (133,6%) и Польше (132,1%). Это говорит о быстром восстановлении и росте интереса к этим странам среди туристов.

- **Страны с умеренным ростом** (около 110–130% к 2022 году): Италия, Великобритания, Нидерланды, Австрия, Мексика, Испания, Греция, Португалия, Египет, Саудовская Аравия, ОАЭ, Румыния.

- **Некоторые страны показывают нестабильную динамику**: например, по Хорватии и Турции отсутствуют данные за отдельные годы, что может быть связано с особенностями сбора статистики или временными ограничениями на въезд.

- **Восстановление туризма после пандемии продолжается**: большинство стран демонстрируют положительную динамику и значительный рост турпотока в 2023–2024 годах по сравнению с 2022 годом.

- **Лидеры по темпам восстановления** — Канада, США, Германия и Польша, что может быть связано с активной политикой по привлечению туристов и снятием ограничений.

**Общий вывод:**  
Крупнейшие туристические направления мира (Франция, Испания, США, Италия, Турция) сохраняют лидерство, а наибольший прирост турпотока наблюдается в странах Северной Америки и Центральной Европы. Туристическая отрасль продолжает активно восстанавливаться после пандемии, однако темпы роста и абсолютные значения различаются в зависимости от страны.


4. Проверка пропусков и аномалий

In [ ]:
query = '''
SELECT country, arrivals_2022, arrivals_2023, arrivals_2024
FROM most_visited_countries
WHERE arrivals_2022 IS NULL
   OR arrivals_2023 IS NULL
   OR arrivals_2024 IS NULL
   OR arrivals_2022 = 0
   OR arrivals_2023 = 0
   OR arrivals_2024 = 0
'''
print(pd.read_sql(query, conn))


              country  arrivals_2022  arrivals_2023  arrivals_2024
0               India            NaN            NaN           17.9
1               China            NaN            NaN           65.7
2           Indonesia            NaN            NaN           15.5
3            Pakistan            NaN            NaN            NaN
4             Nigeria            NaN            NaN            NaN
..                ...            ...            ...            ...
181            Monaco            NaN            NaN            NaN
182  Marshall Islands            NaN            NaN            NaN
183        San Marino            NaN            NaN            NaN
184             Palau            NaN            NaN            NaN
185            Tuvalu            NaN            NaN            NaN

[186 rows x 4 columns]
